# Машинное обучение с подкреплением. МФТИ: МТИИ 2020-2021.
## Семинар 2: Динамическое программирование. 

Рассмотрим алгоритм итерации по оценкам состояния $V$ (Value Iteration):
    $$V_{(i+1)}(s) = \max_a \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')]$$
На основе оценки $V_i$ можно посчитать функцию оценки $Q_i$ действия $a$ в состоянии $s$:
$$Q_i(s, a) = \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')]$$
$$V_{(i+1)}(s) = \max_a Q_i(s,a)$$

Зададим напрямую модель MDP с картинки:
<img src="https://camo.githubusercontent.com/44c1a4d0063e3954f9eb6686a5b7b9763dadd54a/68747470733a2f2f75706c6f61642e77696b696d656469612e6f72672f77696b6970656469612f636f6d6d6f6e732f612f61642f4d61726b6f765f4465636973696f6e5f50726f636573732e737667" caption="Марковский процесс принятия решений" style="width: 400px;">

In [180]:
try:
    import colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !wget https://raw.githubusercontent.com/Tviskaron/mipt/master/2019/RL/02/mdp.py

In [181]:
transition_probs = {
  's0':{
    'a0': {'s0': 0.5, 's2': 0.5},
    'a1': {'s2': 1}
  },
  's1':{
    'a0': {'s0': 0.7, 's1': 0.1, 's2': 0.2},
    'a1': {'s1': 0.95, 's2': 0.05}
  },
  's2':{
    'a0': {'s0': 0.4, 's2': 0.6},
    'a1': {'s0': 0.3, 's1': 0.3, 's2':0.4}
  }
}
rewards = {
  's1': {'a0': {'s0': +5}},
  's2': {'a1': {'s0': -1}}
}

from mdp import MDP
import numpy as np
mdp = MDP(transition_probs, rewards, initial_state='s0')

Теперь мы можем использовать это MDP, как и любое другое gym окружение:

In [182]:
state = mdp.reset()
print('initial state =', state)
next_state, reward, done, info = mdp.step('a1')
print(f'next_state ={next_state}, reward = {reward}, done = {done}')

Также, помимо стандартных методов, есть дополнительные, которые пригодятся нам для реализации метода итерации по полезностям.

In [183]:
print("all_states =", mdp.get_all_states())
print("possible_actions('s1') = ", mdp.get_possible_actions('s1'))
print("next_states('s1', 'a0') = ", mdp.get_next_states('s1', 'a0'))
print("reward('s1', 'a0', 's0') = ",mdp.get_reward('s1', 'a0', 's0'))
print("transition_prob('s1', 'a0', 's0') = ",
      mdp.get_transition_prob('s1', 'a0', 's0'))

### Задание 1

Теперь реализуем алгоритм итерации по полезностям, чтобы решить этот вручную заданный MDP. Псевдокод алгоритма:

---

`1.` Инициализируем $V^{(0)}(s)=0$, для всех $s$

`2.` For $i=0, 1, 2, \dots$
 
`3.` $ \quad V_{(i+1)}(s) = \max_a \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')]$, для всех $s$

---

Вначале вычисляем оценку состояния-действия:
$$Q_i(s, a) = \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')]$$

In [184]:
def get_action_value(mdp, state_values, state, action, 
                     gamma):
    """ Вычислеям Q(s,a) по формуле выше """
    # вычислеяем оценку состояния
    # Q = 
    ####### Здесь ваш код ########      
    raise NotImplementedError      
    ##############################
    
    return Q

In [185]:
test_Vs = {s: i for i, s in enumerate(sorted(mdp.get_all_states()))}
assert np.isclose(get_action_value(mdp, test_Vs, 's2', 'a1', 0.9), 0.69)
assert np.isclose(get_action_value(mdp, test_Vs, 's1', 'a0', 0.9), 3.95)

Теперь оцениваем полезность самого состояния, для этого мы можем использовать предыдущий метод:

$$V_{(i+1)}(s) = \max_a \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')] = \max_a Q_i(s,a)$$

In [186]:
def get_new_state_value(mdp, state_values, state, gamma):
    """ Считаем следующее V(s) по формуле выше."""
    if mdp.is_terminal(state): 
        return 0
    # V = 
    ####### Здесь ваш код ######## 
    raise NotImplementedError 
    ##############################
    
    return V

In [187]:
test_Vs_copy = dict(test_Vs)
assert np.isclose(get_new_state_value(mdp, test_Vs, 's0', 0.9), 1.8)
assert np.isclose(get_new_state_value(mdp, test_Vs, 's2', 0.9), 1.08)
assert np.isclose(get_new_state_value(mdp, {'s0': -1e10, 's1': 0, 's2': -2e10}, 's0', 0.9), -13500000000.0), \
   "Убедитесь, что вы правильно обрабатываете отрицательные значения Q произвольной величины."
assert test_Vs == test_Vs_copy, "Убедитесь, что вы не изменяете state_values в функции get_new_state_value"

Теперь создаем основной цикл итерационного оценки полезности состояний с критерием остановки, который проверяет насколько изменились полезности.

In [188]:
def value_iteration(mdp, state_values=None,
    gamma = 0.9, num_iter = 1000, min_difference = 1e-5):
    """ выполняет num_iter шагов итерации по значениям"""
    # инициализируем V(s)
    state_values = state_values or \
    {s : 0 for s in mdp.get_all_states()}
    
    for i in range(num_iter):
        # Вычисляем новые полезности состояний, 
        # используя функции, определенные выше. 
        # Должен получиться словарь {s: new_V(s)}
        # new_state_values = 
        ####### Здесь ваш код ########
        raise NotImplementedError
        ##############################
        
        assert isinstance(new_state_values, dict)

        # Считаем разницу
        diff =  max(abs(new_state_values[s] - state_values[s]) for s in mdp.get_all_states())
        
        print("iter %4i | diff: %6.5f | V(start): %.3f "%
          (i, diff, new_state_values[mdp._initial_state]))
        
        state_values = new_state_values
        if diff < min_difference:
            print("Принято! Алгоритм сходится!")
            break
            
    return state_values

state_values = value_iteration(mdp,
        num_iter = 100, min_difference = 0.001)

In [189]:
print("Final state values:", state_values)

assert abs(state_values['s0'] - 3.781) < 0.01
assert abs(state_values['s1'] - 7.294) < 0.01
assert abs(state_values['s2'] - 4.202) < 0.01

По найденным полезностям и зная модель переходов легко найти оптимальную стратегию:
$$\pi^*(s) = argmax_a \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')] = argmax_a Q_i(s,a)$$

In [190]:
def get_optimal_action(mdp, state_values, state, 
                       gamma=0.9):
    """ Находим оптимальное действие, используя формулу выше. """
    if mdp.is_terminal(state): return None
    
    actions = mdp.get_possible_actions(state)
    # выбираем лучшее действие
    # i = 
    ####### Здесь ваш код ########
    raise NotImplementedError
    ##############################
    
    return actions[i]

In [191]:
assert get_optimal_action(mdp, state_values, 's0', 0.9) == 'a1'
assert get_optimal_action(mdp, state_values, 's1', 0.9) == 'a0'
assert get_optimal_action(mdp, state_values, 's2', 0.9) == 'a1'

assert get_optimal_action(mdp, {'s0': -1e10, 's1': 0, 's2': -2e10}, 's0', 0.9) == 'a0', \
    "Убедитесь, что вы правильно обрабатываете отрицательные значения Q произвольной величины."
assert get_optimal_action(mdp, {'s0': -2e10, 's1': 0, 's2': -1e10}, 's0', 0.9) == 'a1', \
    "Убедитесь, что вы правильно обрабатываете отрицательные значения Q произвольной величины."

In [192]:
# Проверим среднее вознаграждение агента

s = mdp.reset()
rewards = []
for _ in range(10000):
    s, r, done, _ = mdp.step(get_optimal_action(mdp, state_values, s, 0.9))
    rewards.append(r)

print("average reward: ", np.mean(rewards))

assert(0.40 < np.mean(rewards) < 0.55)

### Задание 2

Теперь проверим работу итерации по ценностям на классической задаче FrozenLake.

In [193]:
from mdp import FrozenLakeEnv
mdp = FrozenLakeEnv(slip_chance=0)

mdp.render()
state_values = value_iteration(mdp)

Визуализируем нашу стратегию.

In [194]:
def draw_policy(mdp, state_values, gamma=0.9):
    """функция визуализации стратегии"""
    plt.figure(figsize=(3, 3))
    h, w = mdp.desc.shape
    states = sorted(mdp.get_all_states())
    V = np.array([state_values[s] for s in states])
    Pi = {
        s: get_optimal_action(mdp, state_values, s, gamma)
        for s in states}
    plt.imshow(V.reshape(w, h),
               cmap='gray', interpolation='none',
               clim=(0, 1))
    ax = plt.gca()
    ax.set_xticks(np.arange(h) - .5)
    ax.set_yticks(np.arange(w) - .5)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    Y, X = np.mgrid[0:4, 0:4]
    a2uv = {'left': (-1, 0), 'down': (0, -1),
            'right': (1, 0), 'up': (-1, 0)}
    for y in range(h):
        for x in range(w):
            plt.text(x, y, str(mdp.desc[y, x].item()),
                     color='g', size=12,
                     verticalalignment='center',
                     horizontalalignment='center',
                     fontweight='bold')
            a = Pi[y, x]
            if a is None: continue
            u, v = a2uv[a]
            plt.arrow(x, y, u * .3, -v * .3,
                      color='m', head_width=0.1,
                      head_length=0.1)
    plt.grid(color='b', lw=2, ls='-')
    plt.show()


In [195]:
from IPython.display import clear_output
from time import sleep
import matplotlib.pyplot as plt
%matplotlib inline
mdp = FrozenLakeEnv(map_name='8x8',slip_chance=0.1)
state_values = {s : 0 for s in mdp.get_all_states()}

for i in range(30):
    clear_output(True)
    print("after iteration %i"%i)
    state_values = value_iteration(mdp, 
                            state_values, num_iter=1)
    draw_policy(mdp, state_values)
    sleep(0.5)

Посмотрим на оптимальную стратегию:

In [196]:
s = mdp.reset()
mdp.render()
for t in range(100):
    a = get_optimal_action(mdp, state_values, s, 0.9)
    print(a, end='\n\n')
    s, r, done, _ = mdp.step(a)
    mdp.render()
    if done:
        break

Тестируем на более сложном варианте окружения:

In [197]:
mdp = FrozenLakeEnv(slip_chance=0.2, map_name='8x8')
state_values = value_iteration(mdp)

total_rewards = []
for game_i in range(1000):
    s = mdp.reset()
    rewards = []
    for t in range(100):
        # выполняем оптимальное действие в окружении
 
        s, r, done, _ = mdp.step(get_optimal_action(mdp, state_values, s, 0.9))
       
        rewards.append(r)
        if done: break
    total_rewards.append(np.sum(rewards))
    
print("Cреднее вознаграждение:", np.mean(total_rewards))
assert(0.6 <= np.mean(total_rewards) <= 0.8)
print("Принято!")

### Задание 3

Теперь рассмотрим алгоритм итерации по стратегиям (PI, policy iteration):

---
Initialize $\pi_0$   `// случайно`

For $n=0, 1, 2, \dots$
- Считаем функцию $V^{\pi_{n}}$
- Используя $V^{\pi_{n}}$, считаем функцию $Q^{\pi_{n}}$
- Получаем новую стратегию: $\pi_{n+1}(s) = \operatorname*{argmax}_a Q^{\pi_{n}}(s,a)$
---

PI включает в себя оценку полезности состояния, как внутренний шаг.

Вначале оценим полезности, используя текущую стратегию:
$$V^{\pi}(s) = \sum_{s'} P(s,\pi(s),s')[ R(s,\pi(s),s') + \gamma V^{\pi}(s')]$$
    Мы будем искать точное решение, хотя могли использовать и предыдущий итерационный подход. Для этого будем решать систему линейных уравнений относительно $V^{\pi}(s_i)$ с помощью np.linalg.solve.

In [198]:
from numpy.linalg import solve

def compute_vpi(mdp, policy, gamma):
    """
    Считем V^pi(s) для всех состояний, согласно стратегии.
    :param policy: словарь состояние->действие {s : a}
    :returns: словарь {state : V^pi(state)}
    """
    states = mdp.get_all_states()
    A, b = [], []
    for i, state in enumerate(states):
        if state in policy:
            a = policy[state]
            # формируем матрицу A (... A.append(...))
            ####### Здесь ваш код ########             
            raise NotImplementedError             
            ##############################
            
            # и вектор b (b.append(...))
            ####### Здесь ваш код ########
            raise NotImplementedError
            ##############################
            
        else:
            # формируем матрицу A (... A.append(...))
            ####### Здесь ваш код ########
            raise NotImplementedError
            ##############################
            
            # вектор b (b.append(...))
            ####### Здесь ваш код ########
            raise NotImplementedError
            ##############################
            
    A = np.array(A)
    b = np.array(b)
    
    values = solve(A, b)
    
    state_values = {states[i] : values[i] 
                    for i in range(len(states))}
    return state_values

In [199]:
transition_probs = {
    's0': {
        'a0': {'s0': 0.5, 's2': 0.5},
        'a1': {'s2': 1}
    },
    's1': {
        'a0': {'s0': 0.7, 's1': 0.1, 's2': 0.2},
        'a1': {'s1': 0.95, 's2': 0.05}
    },
    's2': {
        'a0': {'s0': 0.4, 's1': 0.6},
        'a1': {'s0': 0.3, 's1': 0.3, 's2': 0.4}
    }
}
rewards = {
    's1': {'a0': {'s0': +5}},
    's2': {'a1': {'s0': -1}}
}
mdp = MDP(transition_probs, rewards, initial_state='s0')

gamma = 0.9

test_policy = {
    s: np.random.choice(mdp.get_possible_actions(s))
    for s in mdp.get_all_states()}
new_vpi = compute_vpi(mdp, test_policy, gamma)

print(new_vpi)
assert type(new_vpi) is dict, \
    "функция compute_vpi должна возвращать словарь \
    {состояние s : V^pi(s) }"

Теперь обновляем стратегию на основе новых значений полезностей:

In [200]:
def compute_new_policy(mdp, vpi, gamma):
    """
    Рассчитываем новую стратегию
    :param vpi: словарь {state : V^pi(state) }
    :returns: словарь {state : оптимальное действие}
    """
    Q = {}
    for state in mdp.get_all_states():
        Q[state] = {}
        for a in mdp.get_possible_actions(state):
            values = []
            for next_state in mdp.get_next_states(state,
                                                  a):
                r = mdp.get_reward(state, a, next_state)
                p = mdp.get_transition_prob(state, a,
                                            next_state)
                # values.append(...)
                ####### Здесь ваш код ########
                raise NotImplementedError
                ##############################
                
            Q[state][a] = sum(values)

    policy = {}
    for state in mdp.get_all_states():
        actions = mdp.get_possible_actions(state)
        if actions:
            # выбираем оптимальное действие в state
            # policy[state] = ... 
            ####### Здесь ваш код ########
            raise NotImplementedError
            ##############################
            
    return policy

In [201]:
new_policy = compute_new_policy(mdp, new_vpi, gamma)

print(new_policy)

assert type(new_policy) is dict, \
"функция compute_new_policy должна возвращать словарь \
{состояние s: оптимальное действие}"

Собираем все в единый цикл:

In [202]:
def policy_iteration(mdp, policy=None, gamma = 0.9, 
                 num_iter = 1000, min_difference = 1e-5):
    """ 
    Запускаем цикл итерации по стратегиям 
    Если стратегия не определена, задаем случайную
    """
    for i in range(num_iter):
        if not policy:
            policy = {}
            for s in mdp.get_all_states():
                if mdp.get_possible_actions(s):
                    policy[s] = np.random \
                    .choice(mdp.get_possible_actions(s))
        # state_values = 
        ####### Здесь ваш код ########
        raise NotImplementedError
        ##############################
        
        #policy =
        ####### Здесь ваш код ########
        raise NotImplementedError
        ##############################        
    return state_values, policy

Тестируем на FrozenLake.

In [20]:
mdp = FrozenLakeEnv(slip_chance=0.1)
state_values, policy = policy_iteration(mdp)

total_rewards = []
for game_i in range(1000):
    s = mdp.reset()
    rewards = []
    for t in range(100):
        s, r, done, _ = mdp.step(policy[s])
        rewards.append(r)
        if done: break
    total_rewards.append(np.sum(rewards))
    
print("average reward: ", np.mean(total_rewards))
assert(0.8 <= np.mean(total_rewards) <= 0.95)
print("Принято!")

In [5]:


;